In [70]:
import nilearn
import numpy as np
import pandas as pd
import os
import hcp_utils as hcp
import nibabel as nib
import json
from nilearn import datasets
from nilearn.maskers import NiftiLabelsMasker
from nilearn import signal
from sklearn.impute import SimpleImputer

## Dataset Class

In [65]:
class RawDataset():
    def __init__(self, BIDS_path):
        self.BIDS_path = BIDS_path
        if self.BIDS_path is not None:
            pass
        else:
            raise ValueError("The path to the dataset in BIDS format must be specified (BIDS_path).")
        self.data_description_path = self.BIDS_path + '/dataset_description.json'
        self.participant_data_path = self.BIDS_path + '/participants.tsv'
        self._participant_data = pd.read_csv(self.participant_data_path, sep = '\t')
        self._name = None
        self._data_description = None
        self._subjects = None
        self._group = None

    @property
    def participant_data(self):
        if self._participant_data is None:
            self._participant_data = pd.read_csv(self.participant_data_path, sep = '\t')
        return self._participant_data

    @property
    def subjects(self):
        if self._subjects is None:
            self._subjects = self._participant_data['participant_id'].values
            self._subjects = np.array([i[4:] for i in self._subjects])
        return self._subjects

    @property
    def group(self):
        if self._group is None:
            self._group = np.unique(self._participant_data['group'].values)
        return self._group
    
    @property
    def data_description(self):
        if self._data_description is None:
            self._data_description = json.load(open(self.data_description_path))
        return self._data_description

    @property
    def name(self):
        if self._name is None:
            self._name = self.data_description['Name']
        return self._name
    
    def __repr__(self):
        return f'Dataset(Name={self.name},\nGroup(s)={self.group},\nSubjects={self.subjects},\nData_Path={self.BIDS_path})'


In [145]:
class FmriPreppedDataSet(RawDataset):

    def __init__(self, BIDS_path):
        super().__init__(BIDS_path)
        self.data_path = self.BIDS_path + '/derivatives'
        self.data_path = self._find_sub_dirs()
    def __repr__(self):
        return f'Dataset(Group(s)={self.group},\n Subjects={self.subjects},\n Data_Path={self.data_path})'
    
    def _find_sub_dirs(self):
        path_not_found = True
        while path_not_found:
            subdirs = os.listdir(self.data_path)
            for subdir in subdirs:
                if any(subdir.startswith('sub-') for subdir in subdirs):
                        path_not_found = False
                else:
                    if os.path.isdir(os.path.join(self.data_path, subdir)):
                        self.data_path = os.path.join(self.data_path, subdir)
        return self.data_path
    
    def get_ts_paths(self, subject): # needs to beadaptred to multiple sessions
        subject_dir = os.path.join(self.data_path, f'sub-{subject}', 'func')
        ts_paths = [f'{subject_dir}/{i}' for i in os.listdir(subject_dir) if i.endswith('MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz')] #sub-01_task-rest_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz
        return ts_paths
    
    def get_sessions(self, subject):
        subject_dir = f'{self.data_path}/sub-{subject}'
        subdirs = os.listdir(subject_dir)
        session_names = []
        for subdir in subdirs:
            if subdir.startswith('ses-'):
                session_names.append(subdir[4:])
        return session_names
    
    def _impute_nans_confounds(self, dataframe, pick_confounds = None):
        imputer = SimpleImputer(strategy='mean')
        if pick_confounds is None:
            pick_confounds = np.loadtxt('PyConn/PyConn/preprocessing/default_confounds.txt', dtype = 'str')
        if isinstance(pick_confounds, (list, np.ndarray)):
            df_no_nans = pd.DataFrame(imputer.fit_transform(dataframe), columns=dataframe.columns)[pick_confounds]
        else:
            df_no_nans = pd.DataFrame(imputer.fit_transform(dataframe), columns=dataframe.columns)
        return df_no_nans
    
    def get_confounds(self, subject, no_nans = True, pick_confounds = None):
        if pick_confounds == None:
            pick_confounds = np.loadtxt('PyConn/PyConn/preprocessing/default_confounds.txt', dtype = 'str')

        subject_dir = os.path.join(self.data_path, f'sub-{subject}')
        session_names = self.get_sessions(subject)

        if len(session_names) != 0:
            confound_paths = []
            confound_list = []
            for session_name in session_names:
                session_dir = os.path.join(subject_dir, f'ses-{session_name}', 'func')
                if os.path.exists(session_dir):
                    confound_files = [os.path.join(session_dir, f) for f in os.listdir(session_dir) if f.endswith('confounds_timeseries.tsv')]
                    confound_paths.extend(confound_files)
                    
            if no_nans == True:
                for confounds_path in confound_paths:
                    confounds = pd.read_csv(confounds_path, sep = '\t')
                    confounds = self._impute_nans_confounds(confounds)
                    confound_list.append(confounds)
            else:
                for confounds_path in confound_paths:
                    confounds = pd.read_csv(confounds_path, sep = '\t')[pick_confounds]
                    confound_list.append(confounds)
        else:
            func_dir = os.path.join(subject_dir, "func", "")
            confound_files = [os.path.join(func_dir, f) for f in os.listdir(func_dir) if f.endswith('confounds_timeseries.tsv')]
            if no_nans == True:
                confound_list = [self._impute_nans_confounds(pd.read_csv(i, sep = '\t'), pick_confounds) for i in confound_files]
            else:
                confound_list = [pd.read_csv(i, sep = '\t') for i in confound_files]

        return confound_list
    
    def parcellate(self, subjects = None, parcellation = 'schaefer', n_parcels = 1000, gsr = False):
        if subjects is None:
            subjects = self.subjects
        elif not isinstance(subjects, (list, np.ndarray)):
            subjects = [subjects]
        parc_ts_list = []
        for subject in subjects:
            subject_ts_paths = self.get_ts_paths(subject)
            confounds = self.get_confounds(subject)
            if parcellation == 'schaefer':
                atlas = datasets.fetch_atlas_schaefer_2018(n_rois=n_parcels, yeo_networks=7, resolution_mm=1, base_url= None, resume=True, verbose=1)
            masker =  NiftiLabelsMasker(labels_img=atlas.maps, standardize=True, memory='nilearn_cache', verbose=5)
            for subject_ts, subject_confounds in zip(subject_ts_paths, confounds):
                print(subject_confounds)
                if gsr == False:
                    parc_ts = masker.fit_transform(subject_ts, confounds = subject_confounds.drop("global_signal", axis = 1))
                    parc_ts_list.append(parc_ts)
                else:
                    parc_ts = masker.fit_transform(subject_ts, confounds = subject_confounds)
                    parc_ts_list.append(parc_ts)
        return parc_ts_list
    
    
    

In [146]:
dataset = FmriPreppedDataSet(BIDS_path = '/Users/VictoriaShevchenko/Documents/Python_pour_scientifiques/PyConn/PyConn/data/depression_bezmaternykh')

In [147]:
dataset.parcellate("01")

    global_signal          csf  white_matter   trans_x  trans_x_derivative1  \
0      976.179111  1135.008838   1085.977325 -0.011416            -0.000009   
1      975.927935  1133.604487   1085.407589 -0.000031             0.011386   
2      977.382597  1134.083297   1087.860405 -0.011279            -0.011248   
3      978.010885  1135.508196   1087.812419 -0.011284            -0.000005   
4      976.452445  1134.290473   1086.856216 -0.005518             0.005766   
..            ...          ...           ...       ...                  ...   
95     976.381986  1131.225243   1086.363948 -0.016410             0.000000   
96     977.550177  1132.801917   1084.471060 -0.016441            -0.000031   
97     979.437945  1136.661441   1088.583865 -0.016489            -0.000048   
98     977.163677  1132.414469   1086.640938 -0.016441             0.000048   
99     978.398509  1135.053541   1086.649081 -0.012302             0.004138   

    trans_x_derivative1_power2  trans_x_power2   tr

[array([[-0.34639683,  0.87860435,  0.5665479 , ..., -0.58821434,
          0.7506233 ,  0.38765103],
        [ 1.9045569 ,  1.2167786 ,  1.1301713 , ..., -0.42591262,
          0.88393086,  0.64971757],
        [-1.2253412 , -1.5401714 , -0.8476285 , ...,  0.91552854,
         -1.3531073 , -0.807112  ],
        ...,
        [-0.4155177 , -0.90086687, -0.7759805 , ..., -0.09707052,
         -0.49771705, -0.3119472 ],
        [ 0.45147553, -0.40153697,  0.23378395, ..., -0.61115736,
          0.16013454,  0.6397856 ],
        [ 1.0095524 ,  0.59423864,  0.9778209 , ...,  0.03881342,
         -0.62739056, -0.8561374 ]], dtype=float32)]